# Average Probabilities per Disease Over Various Models


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
columns = ['prob_acne_comedos', 'prob_acne_cystic', 'prob_acne_excoriated',
       'prob_acne_mixed', 'prob_acne_scars', 'prob_actinic_keratosis',
       'prob_alopecia_androgenic', 'prob_alopecia_areata',
       'prob_atopic_dermatitis', 'prob_cheilitis_eczematous',
       'prob_chronic_hand_eczema', 'prob_dyshidrosis', 'prob_folliculitis',
       'prob_genital_warts', 'prob_granuloma_annulare', 'prob_herpes_simplex',
       'prob_intertrigo', 'prob_keratosis_pilaris',
       'prob_lichen_simplex_chronicus', 'prob_melanonychia', 'prob_melasma',
       'prob_molluscum', 'prob_nevus', 'prob_nummular_eczema',
       'prob_peri_oral_dermatitis', 'prob_pityriasis_rosae',
       'prob_plane_warts', 'prob_prurigo_nodularis', 'prob_psoriasis_guttate',
       'prob_psoriasis_pustular_palmoplantar', 'prob_psoriasis_vulgar',
       'prob_rosacea_erythemato_telangiectasique', 'prob_rosacea_inflammatory',
       'prob_seborrheic_dermatitis', 'prob_seborrheic_keratosis',
       'prob_shingles', 'prob_tinea_corporis', 'prob_tinea_versicolor',
       'prob_urticaria', 'prob_vitiligo', 'prob_vulgar_warts', 'src',
       'label']

In [3]:
classes = ['rosacea_inflammatory', 'atopic_dermatitis', 'rosacea_erythemato_telangiectasique','peri_oral_dermatitis',
                'seborrheic_keratosis','psoriasis_vulgar','seborrheic_dermatitis','nummular_eczema',
                'tinea_versicolor','chronic_hand_eczema','vulgar_warts','folliculitis','alopecia_androgenic',
                'dyshidrosis','nevus','melasma','alopecia_areata','intertrigo','urticaria','vitiligo','keratosis_pilaris',
                'molluscum','cheilitis_eczematous','tinea_corporis','prurigo_nodularis','actinic_keratosis',
                'genital_warts','plane_warts','pityriasis_rosae','melanonychia','psoriasis_pustular_palmoplantar',
                'granuloma_annulare','psoriasis_guttate','lichen_simplex_chronicus','shingles','herpes_simplex',
                'acne_cystic', 'acne_scars', 'acne_excoriated', 'acne_comedos', 'acne_mixed',]

In [4]:
df_all = pd.read_csv('test_Merged_For_Ensemble_probs.csv')

In [5]:
def get_df_per_model(model_name: str, df_all: pd.DataFrame)->pd.DataFrame:
    df = df_all.filter(like=f'{model_name}_')
    df[['src', 'label']] = df_all[['src', 'label']]
    df.columns = df.columns.str.replace(f'{model_name}_', '')
    return df

In [7]:
df_vit = get_df_per_model('vit',df_all )
df_bit =  get_df_per_model('bit',df_all )
df_inception_v4 =  get_df_per_model('inception_v4',df_all )
df_resnext =  get_df_per_model('resnext101_64x4d',df_all )

In [8]:
def get_top_k_classification_report(df_preds: pd.DataFrame, k: int, classes: list):
    """Generate top-3 classification report

    Args:
        df_preds (pd.DataFrame): predictions
        classes (list): list of classes

    Returns:
        dict: top-3 classification report
    """        
    exp_name= 'top'+str(k)+'_prediction'
    print(exp_name)
    prob_columns = ['prob_' + disease for disease in classes]
    preds_list = []
    for i in range(k):
        str_name = 'Pred'+ str(i+1)    
        preds_list.append(str_name)
        df_preds[str_name] =df_preds[prob_columns].apply(lambda x: x.sort_values(ascending=False).index[i].replace('prob_', ''), axis=1)
    df_preds['labels'] = df_preds['label']
    df_preds[exp_name] = df_preds.apply(lambda row: bool(set([row['labels']]).intersection(set(row[preds_list].values))), axis=1).reset_index(drop=True)
    print(f"{exp_name} Accuracy: {df_preds[exp_name].value_counts()[True]/df_preds.shape[0]:0.3f}%.")
    y_pred = df_preds.apply(lambda x : x['labels'] if x[exp_name] else x['Pred1'] , axis=1).to_list()
    y_score = df_preds.apply(lambda x : x['labels'], axis=1).to_list()
    report = classification_report( y_score,y_pred, digits=4, output_dict=True)
    #print(classification_report( y_score,y_pred, digits=4, output_dict=False))
    return pd.DataFrame(report).transpose()

## 1. VIT top-3 Report

In [9]:
get_top_k_classification_report(df_vit, k=3, classes=classes)

top3_prediction
top3_prediction Accuracy: 0.888%.


,f1-score,precision,recall,support
acne_comedos,0.727273,0.769231,0.689655,29.00000
acne_cystic,0.833333,0.875000,0.795455,44.00000
acne_excoriated,0.727273,0.857143,0.631579,19.00000
acne_mixed,0.920635,0.878788,0.966667,180.00000
acne_scars,0.918919,1.000000,0.850000,20.00000
actinic_keratosis,0.937500,1.000000,0.882353,17.00000
alopecia_androgenic,0.953846,1.000000,0.911765,34.00000
alopecia_areata,0.750000,0.818182,0.692308,13.00000
atopic_dermatitis,0.860558,0.892562,0.830769,130.00000
cheilitis_eczematous,0.800000,0.782609,0.818182,22.00000


## 2. BIT top-3 Report

In [10]:
get_top_k_classification_report(df_bit, k=3, classes=classes)

top3_prediction
top3_prediction Accuracy: 0.851%.


,f1-score,precision,recall,support
acne_comedos,0.740741,0.800000,0.689655,29.00000
acne_cystic,0.829268,0.894737,0.772727,44.00000
acne_excoriated,0.606061,0.714286,0.526316,19.00000
acne_mixed,0.877193,0.799087,0.972222,180.00000
acne_scars,0.787879,1.000000,0.650000,20.00000
actinic_keratosis,0.758621,0.916667,0.647059,17.00000
alopecia_androgenic,0.941176,0.941176,0.941176,34.00000
alopecia_areata,0.869565,1.000000,0.769231,13.00000
atopic_dermatitis,0.816176,0.781690,0.853846,130.00000
cheilitis_eczematous,0.782609,0.750000,0.818182,22.00000


## 3. Inception v4 top-3 Report

In [11]:
#get_top_k_classification_report(df_inception_v4, k=3, classes=classes)

## Resnext top-3 Report

In [12]:
get_top_k_classification_report(df_resnext, k=3, classes=classes)


top3_prediction
top3_prediction Accuracy: 0.868%.


,f1-score,precision,recall,support
acne_comedos,0.640000,0.761905,0.551724,29.000000
acne_cystic,0.896552,0.906977,0.886364,44.000000
acne_excoriated,0.666667,0.909091,0.526316,19.000000
acne_mixed,0.896552,0.857868,0.938889,180.000000
acne_scars,0.918919,1.000000,0.850000,20.000000
actinic_keratosis,0.937500,1.000000,0.882353,17.000000
alopecia_androgenic,0.826667,0.756098,0.911765,34.000000
alopecia_areata,0.888889,0.857143,0.923077,13.000000
atopic_dermatitis,0.863636,0.850746,0.876923,130.000000
cheilitis_eczematous,0.775510,0.703704,0.863636,22.000000


In [18]:
def create_merge_file():
    df = df_vit[columns]
    df = pd.concat([df, df_bit[columns]])
    df = pd.concat([df, df_resnext[columns]])
    #df = pd.concat([df, df_inception_v4[columns]])
    #df = pd.concat([df, pd.read_csv(f'gs://oro-ds-test-bucket/sdd_acne_files/mlflow_prod/bcb6acc974e340ba988430e581c54553/artifacts/files/inceptionv4-0-{mode}_predictions.csv')[columns]])
    #df = pd.concat([df, pd.read_csv(f'gs://oro-ds-test-bucket/sdd_acne_files/mlflow_prod/e9e9d6aeb4d04f2ca1437d8f3c3dacd3/artifacts/files/resnext101_64x4d-0-{mode}_predictions.csv')[columns]])
    #df = pd.concat([df, pd.read_csv(f'gs://oro-ds-test-bucket/sdd_acne_files/mlflow_prod/4a4add34e88545d7b963f051234b88dc/artifacts/files/bit-1-{mode}_predictions.csv')[columns]])
    #df = pd.concat([df, pd.read_csv(f'gs://oro-ds-test-bucket/sdd_acne_files/mlflow_prod/0173da554bcc4e52b692c074e3de7597/artifacts/files/inceptionv4-1-test_predictions.csv')[columns]])
    #df = pd.concat([df, pd.read_csv(f'gs://oro-ds-test-bucket/sdd_acne_files/mlflow_prod/bd922485a00d4f6389d6396d990af2dd/artifacts/files/resnext50_32x4d-1-test_predictions.csv')[columns]])

    return df

In [19]:
df_test= create_merge_file()

In [20]:
df  = df_test.groupby(by=['src']).mean()

In [21]:
df = df.merge(df_all[['src', 'label']], on='src', how='left')

In [22]:
get_top_k_classification_report(df, k=3, classes=classes)

top3_prediction
top3_prediction Accuracy: 0.904%.


,f1-score,precision,recall,support
acne_comedos,0.730769,0.826087,0.655172,29.000000
acne_cystic,0.904762,0.950000,0.863636,44.000000
acne_excoriated,0.733333,1.000000,0.578947,19.000000
acne_mixed,0.926316,0.880000,0.977778,180.000000
acne_scars,0.894737,0.944444,0.850000,20.000000
actinic_keratosis,0.903226,1.000000,0.823529,17.000000
alopecia_androgenic,0.985075,1.000000,0.970588,34.000000
alopecia_areata,0.888889,0.857143,0.923077,13.000000
atopic_dermatitis,0.893130,0.886364,0.900000,130.000000
cheilitis_eczematous,0.782609,0.750000,0.818182,22.000000
